In [21]:
# constant
DF_PATH = "../data/processed/1_preprocessed_numeric_df.pkl"


### Libararies 

In [5]:

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import re
import gensim

# preprocess
from sklearn.compose import make_column_transformer , ColumnTransformer
from sklearn.preprocessing import StandardScaler 
#from sklearn.preprocessing import TargetEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso , LassoCV , Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from pandas import option_context

# metric
from sklearn.metrics import make_scorer,mean_squared_error
scoring = make_scorer(mean_squared_error) 

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from autocorrect import Speller
spell = Speller()
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin


In [22]:
#read the dataFrame
# Read Data 
df = pd.read_pickle(DF_PATH)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
1,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
2,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
3,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
4,US,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini


### functions
 - preprocessing for BOW

In [7]:
TEXT_CLEANING_RE = "[^A-Za-z0-9]"
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

# for every row
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(spell(token)))
            else:
                tokens.append(token)
    return " ".join(tokens)


def _pre_all(data,stem = True):
    X = data.apply(lambda x : preprocess(x))
  
    return X
    
pre_all = FunctionTransformer(_pre_all)

### best params of BOW from last tuning
- we can tuning with this task only but still good idea to use thesis hyperparameters.


In [8]:
# best params of last
params = {'regressor': Ridge(),
 'regressor__alpha': 0.0001,
 'vec2': 'passthrough',
 'vectorizer': CountVectorizer(ngram_range=(2, 2)),
 'vectorizer__analyzer': 'word',
 'vectorizer__ngram_range': (1, 1)}

##### params to use in the comming BOW models

### compine non-text model with BOW model
### BOW + numeric

In [23]:
X_train,X_test,y_train,y_test = train_test_split(df[['price','province','description']] , df['points'] ,  random_state=42)

> we will use the best hyperparameters form the previus tuning

In [14]:
params

{'regressor': Ridge(),
 'regressor__alpha': 0.0001,
 'vec2': 'passthrough',
 'vectorizer': CountVectorizer(),
 'vectorizer__analyzer': 'word',
 'vectorizer__ngram_range': (1, 1)}

In [15]:
# prepare the prameters, split the params to the BOW and the regressor parameters
reg_params = {}
BOW_params = params.copy()

# may we use it or not, is not a good practice
reg_params['regressor'] =  BOW_params.pop('regressor')
reg_params['regressor__alpha'] = BOW_params.pop('regressor__alpha')
BOW_params

{'vec2': 'passthrough',
 'vectorizer': CountVectorizer(),
 'vectorizer__analyzer': 'word',
 'vectorizer__ngram_range': (1, 1)}

In [16]:
pipe_text = Pipeline([
    ('preprocess',pre_all),
    ('vectorizer',CountVectorizer()),
    ('vec2' , TfidfTransformer() )
])

# set params
pipe_text.set_params(**BOW_params)

preprocess_num = make_column_transformer(
    (StandardScaler(), ['price']),
    (TargetEncoder(), ['province']),
    remainder = 'drop'
)


# transformer to compine the tow different data and make the true transform.
# define the transformer
class preprocess_comp(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        self.text =  pipe_text.fit(X['description'],y )
        self.num = preprocess_num.fit(X,y)   
        return self
    def transform(self, X):
        X = np.hstack(((self.text.transform(X['description'])).toarray(),(self.num.transform(X))))
        return X

In [24]:
# BOW features

# 2 ways to select best vectorizer, first select any vectorizer then tuning, second take the best vectorizer from the last tuning 
# i prefere the second option

pipe = Pipeline([
    ('preprocess',preprocess_comp()),
     ('regressor', Lasso())
])


# set params
pipe.set_params(**reg_params)


pipe.fit(X_train,y_train)
print(' mean_squared_error on train set = ', mean_squared_error(y_train,pipe.predict(X_train)))
print(' mean_squared_error on test set = ', mean_squared_error(y_test,pipe.predict(X_test)))


 mean_squared_error on train set =  1.1458497679939343
 mean_squared_error on test set =  4.387097338027399
